In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras import metrics
import matplotlib.pyplot as plt
from utilities.rectified_adam import RAdam
import numpy as np
import sys
from utilities.pythonDB import writeToDB, deleteExistingPrimaryKeyDB
from build_augmentor import return_augmented_dataset
%matplotlib inline

In [2]:
def get_random_data(data1, data2, low, high, max_samples=100):
    _, H1, W1, C1 = data1.shape
    _, N = data2.shape
    suff_data1 = np.zeros((max_samples, H1, W1, C1))
    suff_data2 = np.zeros((max_samples, N))
    shuffles = np.random.randint(low, high+1, max_samples)
    for idx in range(shuffles.shape[0]):
        suff_data1[idx] = data1[idx, :, :, :]
        suff_data2[idx] = data2[idx, :]
    return suff_data1, suff_data2

In [3]:
MAX_TRAINING = 20000 #25000
MAX_VALIDATION = 2000 #5000
MAX_TESTING = 4000 #4000
global MAX_EPOCHS, MAX_BATCH_SIZE, architecture, label, keywords
MAX_EPOCHS, MAX_BATCH_SIZE = 15, 1024
architecture = 'LeNet5'
label, keywords = 'final-experiments1', 'benchmark_evaluation'

In [4]:
#Loading CIFAR-10 dataset
cifar = tf.keras.datasets.cifar10 
(x_train, y_train), (x_test, y_test) = cifar.load_data()

# Keeping the data between 0 and 1
x_train_max, x_test_max = x_train / 255.0, x_test / 255.0
y_train_max = to_categorical(y_train, num_classes=10)
y_test_max = to_categorical(y_test, num_classes=10)

train_cutoff = int(x_train_max.shape[0] - MAX_VALIDATION*1.5)

x_train, y_train = get_random_data(x_train_max, y_train_max, 0, train_cutoff, MAX_TRAINING)
x_train, y_train = return_augmented_dataset(x_train, y_train, -1)

np.save('mixed_set/x_train.npy', x_train)
np.save('mixed_set/y_train.npy', y_train)

In [5]:
x_train, y_train = np.load('mixed_set/x_train.npy'), np.load('mixed_set/y_train.npy')
x_val, y_val = get_random_data(x_train_max, y_train_max, train_cutoff+1, x_train_max.shape[0], MAX_VALIDATION)
x_test, y_test = get_random_data(x_test_max, y_test_max, 0, x_test_max.shape[0], MAX_TESTING)
print ("Number of Training Samples: X={}, Y={}".format(x_train.shape[0], y_train.shape[0]))
print ("Number of Validation Samples: X={}, Y={}".format(x_val.shape[0], y_val.shape[0]))
print ("Number of Test Samples: X={}, Y={}".format(x_test.shape[0], y_test.shape[0]))

Number of Training Samples: X=22400, Y=22400
Number of Validation Samples: X=2000, Y=2000
Number of Test Samples: X=4000, Y=4000


In [8]:
class MetricsAfterEachEpoch(tf.keras.callbacks.Callback):
    def on_train_begin(self, scores={}):
        self.loss, self.accuracy, self.mae, self.mse = [], [], [], []
    def on_train_end(self, logs={}):
        return self.loss, self.accuracy, self.mae, self.loss
    def on_epoch_end(self, epoch, scores):
        self.loss.append(scores.get('loss'))
        self.accuracy.append(scores.get('acc'))
        self.mae.append(scores.get('mean_absolute_error'))
        self.loss.append(scores.get('mean_squared_error'))
        return

In [7]:
activation_func, optimizer = 'relu', 'Adam'
MAX_BATCH_SIZE = 256

In [15]:
model = Sequential()

#Instantiating Layer 1
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

# #Instantiating Layer 2
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid', kernel_initializer = 'he_uniform'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
model.add(BatchNormalization())
model.add(Dropout(.2))
# #Instantiating Layer 3
model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid', kernel_initializer = 'he_uniform'))
model.add(BatchNormalization())
model.add(Dropout(.2))
model.add(Flatten())

#Instantiating Layer 4
model.add(Dense(84, activation=activation_func)) 
model.add(BatchNormalization())

#Output Layer
model.add(Dense(10, activation='softmax'))

In [16]:
model.compile(loss="categorical_crossentropy", optimizer=optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
history = model.fit(x_train, y_train, epochs=MAX_EPOCHS, batch_size=1024).history
score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
print ("Accuray = {}".format(score[1]))

Train on 22400 samples
Epoch 1/15
22400/22400 [==============================] - 3s 136us/sample - loss: 2.3604 - acc: 0.1490 - mean_absolute_error: 0.1707 - mean_squared_error: 0.0898
Epoch 2/15
22400/22400 [==============================] - 2s 84us/sample - loss: 2.2046 - acc: 0.1608 - mean_absolute_error: 0.1689 - mean_squared_error: 0.0863
Epoch 3/15
22400/22400 [==============================] - 2s 84us/sample - loss: 2.1564 - acc: 0.1677 - mean_absolute_error: 0.1684 - mean_squared_error: 0.0853
Epoch 4/15
22400/22400 [==============================] - 2s 83us/sample - loss: 2.1396 - acc: 0.1725 - mean_absolute_error: 0.1680 - mean_squared_error: 0.0848
Epoch 5/15
22400/22400 [==============================] - 2s 85us/sample - loss: 2.1286 - acc: 0.1729 - mean_absolute_error: 0.1680 - mean_squared_error: 0.0845
Epoch 6/15
22400/22400 [==============================] - 2s 85us/sample - loss: 2.1195 - acc: 0.1729 - mean_absolute_error: 0.1676 - mean_squared_error: 0.0841
Epoch 7/15

In [ ]:
# Run1 - 0.0890, Run2 -0.1210, Run3-.15, Run4-.4872, Run5, 

In [ ]:
# #39%
# model = Sequential()

# #Instantiating Layer 1
# model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
# model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))
# model.add(BatchNormalization())

# # #Instantiating Layer 2
# model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid', kernel_initializer = 'he_uniform'))
# model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))
# model.add(BatchNormalization())

# # #Instantiating Layer 3
# model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid', kernel_initializer = 'he_uniform'))
# model.add(BatchNormalization())
# model.add(Flatten())

# #Instantiating Layer 4
# model.add(Dense(84, activation=activation_func)) 
# model.add(BatchNormalization())

# #Output Layer
# model.add(Dense(10, activation='softmax'))

In [ ]:
def model_extractor(activation_func):
    # Creating a LeNet5 Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1'))

    # #Instantiating Layer 2
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), name='pool2'))

    # #Instantiating Layer 3
    model.add(Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation=activation_func, padding='valid'))

    model.add(Flatten())

    #Instantiating Layer 4
    model.add(Dense(84, activation=activation_func)) 

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
def diff_optimizer(curr_optimizer, activation_func, label=None, curr_epochs=MAX_EPOCHS):
    model = model_extractor(activation_func)
    model.compile(loss="categorical_crossentropy", optimizer=curr_optimizer, validation_data=(x_val, y_val),
                  metrics=["accuracy", "mae", "mse"])
    history = model.fit(x_train, y_train, epochs=curr_epochs, batch_size=1024).history
    score = model.evaluate(x_test, y_test, batch_size=MAX_BATCH_SIZE)
    
    #Saving the model
    if label == 'RAdam':
        model.save("../../saved_models/{}_{}_{}.hdf5".format('RAdam', activation_func, architecture))
    else: 
        model.save("../../saved_models/{}_{}_{}.hdf5".format(curr_optimizer, activation_func, architecture))
    return score, history

In [ ]:
testing_optimizers = ['RAdam']#['RAdam', 'Adam', 'NAdam', 'SGD']
testing_activations = ['tanh'] #['tanh', 'relu', 'selu']

for optimizer in testing_optimizers:
    for activation in testing_activations:
        loss, accuracy, mae, mse = {}, {}, {}, {}
        train_loss_dict, train_accuracy_dict, train_mae_dict, train_mse_dict = {}, {}, {}, {}

        if optimizer == 'RAdam':
            score_returned, history = diff_optimizer(RAdam(lr=0.005), activation, label='RAdam')
        else:
            score_returned, history = diff_optimizer(optimizer, activation)


        bg = (score_returned[0], score_returned[1], score_returned[2], score_returned[3], history.get('loss'),\
          history.get('accuracy'), history.get('mae'), history.get('mae'))

        #Delete Existing Primary Keys and then write to DB
        deleteExistingPrimaryKeyDB(optimizer, activation, architecture, label, keywords)
        writeToDB(optimizer, activation, architecture, bg, label, keywords)

print ("Completed Execution for architecture={}".format(architecture))

In [ ]:
model.summary